In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data')
data.columns = ['Sample code', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
'Normal Nucleoli', 'Mitoses', 'Class']

In [3]:
data.drop(['Sample code'], axis = 1,inplace=True)
data.replace('?',0,inplace=True)
data['Bare Nuclei'] = data['Bare Nuclei'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Clump Thickness              698 non-null    int64
 1   Uniformity of Cell Size      698 non-null    int64
 2   Uniformity of Cell Shape     698 non-null    int64
 3   Marginal Adhesion            698 non-null    int64
 4   Single Epithelial Cell Size  698 non-null    int64
 5   Bare Nuclei                  698 non-null    int64
 6   Bland Chromatin              698 non-null    int64
 7   Normal Nucleoli              698 non-null    int64
 8   Mitoses                      698 non-null    int64
 9   Class                        698 non-null    int64
dtypes: int64(10)
memory usage: 54.7 KB


In [4]:
values = data.values
imputer = SimpleImputer()
imputeData = imputer.fit_transform(values)

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
normalizedData = scaler.fit_transform(imputeData)

In [6]:
X = normalizedData[:, 0:9]
y = normalizedData[:, 9]

In [8]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
LogReg_clf = LogisticRegression()
Dtree_clf = DecisionTreeClassifier()
svc_clf = SVC()

LogReg_clf.fit(X,y)
Dtree_clf.fit(X,y)
svc_clf.fit(X,y)

SVC()

In [9]:
LogReg_pred = LogReg_clf.predict(X)
Dtree_pred = Dtree_clf.predict(X)
svc_pred = svc_clf.predict(X)

avg_preds = (LogReg_pred + Dtree_pred + svc_pred)//3
acc = r2_score(y, avg_preds)
print(acc)

0.9112741403887885


In [10]:
avg_preds

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 1.

In [12]:
# Voting Ensemble
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection

kfold = model_selection.KFold(n_splits = 10, random_state = 7, shuffle = True)
estimators = []
model1 = LogisticRegression()
estimators.append(('logisitic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.961304347826087
